# Chapter 10 - Data Aggregation & Group Operations

In [2]:
import numpy as np
import pandas as pd

# 10.1 How to Think About Group Operations

In [3]:
df = pd.DataFrame({"key1": ["a", "a", None, "b", "b", "a", None],
                  "key2": pd.Series([1, 2, 1, 2, 1, None, 1], dtype="Int64"),
                  "data1": np.random.standard_normal(7),
                  "data2": np.random.standard_normal(7)})

df

,key1,key2,data1,data2
0,a,1,-1.682966,1.012918
1,a,2,-0.865101,0.536812
2,None,1,0.260731,0.505731
3,b,2,-0.036785,-1.269857
4,b,1,-0.563870,-0.000494
5,a,<NA>,1.150292,1.702814
6,None,1,1.402619,-0.668519


In [4]:
grouped = df["data1"].groupby(df["key1"])
grouped

In [8]:
grouped.mean()

key1
a   -0.209696
b    1.451570
Name: data1, dtype: float64

In [9]:
means = df["data1"].groupby([df["key1"], df["key2"]]).mean()
means

key1  key2
a     1      -0.422223
      2       0.486931
b     1       1.559899
      2       1.343241
Name: data1, dtype: float64

means.unstack()

In [10]:
states = np.array(["OH", "CA", "CA", "OH", "OH", "CA", "OH"])
years = [2005, 2005, 2006, 2005, 2006, 2005, 2006]

In [11]:
df["data1"].groupby([states, years]).mean()

CA  2005   -0.103433
    2006    0.744020
OH  2005    0.460509
    2006    0.347289
Name: data1, dtype: float64

In [12]:
df.groupby("key1").mean()

,key2,data1,data2
key1,,,
a,1.5,-0.209696,-0.055099
b,1.5,1.451570,0.723064


In [13]:
df.groupby("key2").mean()

/var/folders/wh/l_q69dcj75z0c82jy8ry71f40000gn/T/ipykernel_9667/2972670978.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df.groupby("key2").mean()


,data1,data2
key2,,
1,0.254094,-0.085330
2,0.915086,0.755663


In [14]:
df.groupby(["key1", "key2"]).mean()

data1     data2
key1 key2                    
a    1    -0.422223  0.171901
     2     0.486931 -0.049714
b    1     1.559899 -0.114913
     2     1.343241  1.561040

In [15]:
df.groupby(["key1", "key2"]).size()

key1  key2
a     1       1
      2       1
b     1       1
      2       1
dtype: int64

In [16]:
df.groupby("key1", dropna=False).size()

key1
a      3
b      2
NaN    2
dtype: int64

In [17]:
df.groupby(["key1", "key2"], dropna=False).size()

key1  key2
a     1       1
      2       1
      <NA>    1
b     1       1
      2       1
NaN   1       2
dtype: int64

In [18]:
df.groupby("key1").count()

,key2,data1,data2
key1,,,
a,2,3,3
b,2,2,2


### Iterating over Groups

In [19]:
for name, group in df.groupby("key1"):
    print(name)
    print(group)

a
  key1  key2     data1     data2
0    a     1 -0.422223  0.171901
1    a     2  0.486931 -0.049714
5    a  <NA> -0.693796 -0.287483
b
  key1  key2     data1     data2
3    b     2  1.343241  1.561040
4    b     1  1.559899 -0.114913


In [20]:
for (k1, k2), group in df.groupby(["key1", "key2"]):
    print((k1, k2))
    print(group)

('a', 1)
  key1  key2     data1     data2
0    a     1 -0.422223  0.171901
('a', 2)
  key1  key2     data1     data2
1    a     2  0.486931 -0.049714
('b', 1)
  key1  key2     data1     data2
4    b     1  1.559899 -0.114913
('b', 2)
  key1  key2     data1    data2
3    b     2  1.343241  1.56104


In [21]:
pieces = {name:group for name, group in df.groupby("key1")}
pieces["b"]

,key1,key2,data1,data2
3,b,2,1.343241,1.561040
4,b,1,1.559899,-0.114913


In [22]:
grouped = df.groupby({"key1": "key", "key2": "key",
                     "data1": "data", "data2": "data"}, axis="columns")

In [23]:
for group_key, group_values in grouped:
    print(group_key)
    print(group_values)

data
      data1     data2
0 -0.422223  0.171901
1  0.486931 -0.049714
2  0.744020 -1.309470
3  1.343241  1.561040
4  1.559899 -0.114913
5 -0.693796 -0.287483
6 -0.865321  0.911161
key
   key1  key2
0     a     1
1     a     2
2  None     1
3     b     2
4     b     1
5     a  <NA>
6  None     1


### Selecting a Columns or Subset of columns

In [24]:
df.groupby("key1")["data1"]
df.groupby("key1")[["data2"]]

In [25]:
df["data1"].groupby(df["key1"])
df[["data2"]].groupby(df["key1"])

In [26]:
df.groupby(["key1", "key2"])[["data2"]].mean()

data2
key1 key2          
a    1     0.171901
     2    -0.049714
b    1    -0.114913
     2     1.561040

In [27]:
s_grouped = df.groupby(["key1", "key2"])["data2"]
s_grouped

In [28]:
s_grouped.mean()

key1  key2
a     1       0.171901
      2      -0.049714
b     1      -0.114913
      2       1.561040
Name: data2, dtype: float64

### Grouping with Dictionaries & Series

In [29]:
people = pd.DataFrame(np.random.standard_normal((5, 5)),
                     columns=["a", "b", "c", "d", "e"],
                     index=["Joe", "Steve", "Wanda", "Jill", "Trey"])

In [30]:
people.iloc[2:3, [1, 2]] = np.nan
people

,a,b,c,d,e
Joe,1.903681,0.206370,0.401418,0.660380,-0.742178
Steve,-0.147456,0.201236,0.422565,1.466925,1.104602
Wanda,1.962130,NaN,NaN,-0.837574,2.101358
Jill,1.535207,1.052277,2.432277,0.310520,-0.633229
Trey,-1.042599,-0.864726,0.351253,-0.455808,1.603895


In [31]:
mapping = {"a": "red", "b": "red", "c": "blue",
          "d": "blue", "e": "red", "f": "orange"}

In [32]:
by_column = people.groupby(mapping, axis= "columns")

In [33]:
by_column.sum()

,blue,red
Joe,1.061798,1.367873
Steve,1.889490,1.158381
Wanda,-0.837574,4.063488
Jill,2.742797,1.954255
Trey,-0.104556,-0.303430


In [34]:
map_series = pd.Series(mapping)
map_series

a       red
b       red
c      blue
d      blue
e       red
f    orange
dtype: object

In [35]:
people.groupby(map_series, axis="columns").count()

,blue,red
Joe,2,3
Steve,2,3
Wanda,1,2
Jill,2,3
Trey,2,3


### Grouping with Functions

In [36]:
people.groupby(len).sum()

,a,b,c,d,e
3,1.903681,0.206370,0.401418,0.660380,-0.742178
4,0.492608,0.187551,2.783530,-0.145288,0.970666
5,1.814673,0.201236,0.422565,0.629351,3.205960


In [37]:
key_list = ["one", "one", "one", "two", "two"]

In [38]:
people.groupby([len, key_list]).min()

,,a,b,c,d,e
3,one,1.903681,0.206370,0.401418,0.660380,-0.742178
4,two,-1.042599,-0.864726,0.351253,-0.455808,-0.633229
5,one,-0.147456,0.201236,0.422565,-0.837574,1.104602


### Grouping by Index Levels

In [39]:
columns = pd.MultiIndex.from_arrays([["US", "US", "US", "JP", "JP"],
                                    [1, 3, 5, 1, 3]],
                                   names=["cty", "tenor"])

In [40]:
hier_df = pd.DataFrame(np.random.standard_normal((4, 5)), columns=columns)

In [41]:
hier_df

cty          US                            JP          
tenor         1         3         5         1         3
0     -0.857541 -0.708594  1.743774 -0.890468 -0.737195
1     -0.679228  0.269535 -0.911005 -0.065212  1.709049
2     -1.074623 -0.692043  1.137842  0.438169  0.775621
3      0.181234  0.149506 -0.251841 -1.052758  0.043063

# 10.2 Data Aggregation

In [42]:
df

,key1,key2,data1,data2
0,a,1,-0.422223,0.171901
1,a,2,0.486931,-0.049714
2,None,1,0.744020,-1.309470
3,b,2,1.343241,1.561040
4,b,1,1.559899,-0.114913
5,a,<NA>,-0.693796,-0.287483
6,None,1,-0.865321,0.911161


In [43]:
grouped = df.groupby("key1")

In [44]:
grouped["data1"].nsmallest(2)

key1   
a     5   -0.693796
      0   -0.422223
b     3    1.343241
      4    1.559899
Name: data1, dtype: float64

In [13]:
def peak_to_peak(arr):
    return arr.max() - arr.min()

In [14]:
grouped.agg(peak_to_peak)

/var/folders/wh/l_q69dcj75z0c82jy8ry71f40000gn/T/ipykernel_16188/4193407341.py:1: FutureWarning: Dropping invalid columns in DataFrameGroupBy.agg is deprecated. In a future version, a TypeError will be raised. Before calling .agg, select only columns which should be valid for the function.
  grouped.agg(peak_to_peak)


total_bill   tip  size   tip_pct
day  smoker                                  
Fri  No           10.29  2.00     1  0.067349
     Yes          34.42  3.73     3  0.159925
Sat  No           41.08  8.00     3  0.235193
     Yes          47.74  9.00     4  0.290095
Sun  No           39.40  4.99     4  0.193226
     Yes          38.10  5.00     3  0.644685
Thur No           33.68  5.45     5  0.193350
     Yes          32.77  3.00     2  0.151240

In [15]:
grouped.describe()

total_bill                                                        \
                 count       mean        std    min     25%     50%      75%   
day  smoker                                                                    
Fri  No            4.0  18.420000   5.059282  12.46  15.100  19.235  22.5550   
     Yes          15.0  16.813333   9.086388   5.75  11.690  13.420  18.6650   
Sat  No           45.0  19.661778   8.939181   7.25  14.730  17.820  20.6500   
     Yes          42.0  21.276667  10.069138   3.07  13.405  20.390  26.7925   
Sun  No           57.0  20.506667   8.130189   8.77  14.780  18.430  25.0000   
     Yes          19.0  24.120000  10.442511   7.25  17.165  23.100  32.3750   
Thur No           45.0  17.113111   7.721728   7.51  11.690  15.950  20.2700   
     Yes          17.0  19.190588   8.355149  10.34  13.510  16.470  19.8100   

                     tip            ...  size      tip_pct            \
               max count      mean  ...   75%  max   count      mean   
day  smoker                         ...                                
Fri  No      22.75   4.0  2.812500  ...  2.25  3.0     4.0  0.151650   
     Yes     40.17  15.0  2.714000  ...  2.00  4.0    15.0  0.174783   
Sat  No      48.33  45.0  3.102889  ...  3.00  4.0    45.0  0.158048   
     Yes     50.81  42.0  2.875476  ...  3.00  5.0    42.0  0.147906   
Sun  No      48.17  57.0  3.167895  ...  4.00  6.0    57.0  0.160113   
     Yes     45.35  19.0  3.516842  ...  3.00  5.0    19.0  0.187250   
Thur No      41.19  45.0  2.673778  ...  2.00  6.0    45.0  0.160298   
     Yes     43.11  17.0  3.030000  ...  2.00  4.0    17.0  0.163863   

                                                                         
                  std       min       25%       50%       75%       max  
day  smoker                                                              
Fri  No      0.028123  0.120385  0.137239  0.149241  0.163652  0.187735  
     Yes     0.051293  0.103555  0.133739  0.173913  0.209240  0.263480  
Sat  No      0.039767  0.056797  0.136240  0.150152  0.183915  0.291990  
     Yes     0.061375  0.035638  0.091797  0.153624  0.190502  0.325733  
Sun  No      0.042347  0.059447  0.139780  0.161665  0.185185  0.252672  
     Yes     0.154134  0.065660  0.097723  0.138122  0.215325  0.710345  
Thur No      0.038774  0.072961  0.137741  0.153492  0.184843  0.266312  
     Yes     0.039389  0.090014  0.148038  0.153846  0.194837  0.241255  

[8 rows x 32 columns]

### Column-Wise & Multiple Function Application

In [5]:
tips = pd.read_csv("examples/tips.csv")

In [6]:
tips.head()

,total_bill,tip,smoker,day,time,size
0,16.99,1.01,No,Sun,Dinner,2
1,10.34,1.66,No,Sun,Dinner,3
2,21.01,3.50,No,Sun,Dinner,3
3,23.68,3.31,No,Sun,Dinner,2
4,24.59,3.61,No,Sun,Dinner,4


In [7]:
tips["tip_pct"] = tips["tip"] / tips["total_bill"]

In [8]:
tips.head()

,total_bill,tip,smoker,day,time,size,tip_pct
0,16.99,1.01,No,Sun,Dinner,2,0.059447
1,10.34,1.66,No,Sun,Dinner,3,0.160542
2,21.01,3.50,No,Sun,Dinner,3,0.166587
3,23.68,3.31,No,Sun,Dinner,2,0.139780
4,24.59,3.61,No,Sun,Dinner,4,0.146808


In [9]:
grouped = tips.groupby(["day", "smoker"])

In [10]:
grouped_pct = grouped["tip_pct"]

In [11]:
grouped_pct.agg("mean")

day   smoker
Fri   No        0.151650
      Yes       0.174783
Sat   No        0.158048
      Yes       0.147906
Sun   No        0.160113
      Yes       0.187250
Thur  No        0.160298
      Yes       0.163863
Name: tip_pct, dtype: float64

In [16]:
grouped_pct.agg(["mean", "std", peak_to_peak])

mean       std  peak_to_peak
day  smoker                                  
Fri  No      0.151650  0.028123      0.067349
     Yes     0.174783  0.051293      0.159925
Sat  No      0.158048  0.039767      0.235193
     Yes     0.147906  0.061375      0.290095
Sun  No      0.160113  0.042347      0.193226
     Yes     0.187250  0.154134      0.644685
Thur No      0.160298  0.038774      0.193350
     Yes     0.163863  0.039389      0.151240

In [17]:
grouped_pct.agg([("average", "mean"), ("stdev", np.std)])

average     stdev
day  smoker                    
Fri  No      0.151650  0.028123
     Yes     0.174783  0.051293
Sat  No      0.158048  0.039767
     Yes     0.147906  0.061375
Sun  No      0.160113  0.042347
     Yes     0.187250  0.154134
Thur No      0.160298  0.038774
     Yes     0.163863  0.039389

In [18]:
functions = ["count", "mean", "max"]

In [19]:
result = grouped[["tip_pct", "total_bill"]].agg(functions)
result

tip_pct                     total_bill                  
              count      mean       max      count       mean    max
day  smoker                                                         
Fri  No           4  0.151650  0.187735          4  18.420000  22.75
     Yes         15  0.174783  0.263480         15  16.813333  40.17
Sat  No          45  0.158048  0.291990         45  19.661778  48.33
     Yes         42  0.147906  0.325733         42  21.276667  50.81
Sun  No          57  0.160113  0.252672         57  20.506667  48.17
     Yes         19  0.187250  0.710345         19  24.120000  45.35
Thur No          45  0.160298  0.266312         45  17.113111  41.19
     Yes         17  0.163863  0.241255         17  19.190588  43.11

In [59]:
result["tip_pct"]

count      mean       max
day  smoker                           
Fri  No          4  0.151650  0.187735
     Yes        15  0.174783  0.263480
Sat  No         45  0.158048  0.291990
     Yes        42  0.147906  0.325733
Sun  No         57  0.160113  0.252672
     Yes        19  0.187250  0.710345
Thur No         45  0.160298  0.266312
     Yes        17  0.163863  0.241255

In [22]:
ftuples = [("Average", "mean"), ("Variance", np.var)]

In [23]:
grouped[["tip_pct", "total_bill"]].agg(ftuples)

tip_pct           total_bill            
              Average  Variance    Average    Variance
day  smoker                                           
Fri  No      0.151650  0.000791  18.420000   25.596333
     Yes     0.174783  0.002631  16.813333   82.562438
Sat  No      0.158048  0.001581  19.661778   79.908965
     Yes     0.147906  0.003767  21.276667  101.387535
Sun  No      0.160113  0.001793  20.506667   66.099980
     Yes     0.187250  0.023757  24.120000  109.046044
Thur No      0.160298  0.001503  17.113111   59.625081
     Yes     0.163863  0.001551  19.190588   69.808518

In [24]:
grouped.agg({"tip": np.max, "size": "sum"})

tip  size
day  smoker             
Fri  No       3.50     9
     Yes      4.73    31
Sat  No       9.00   115
     Yes     10.00   104
Sun  No       6.00   167
     Yes      6.50    49
Thur No       6.70   112
     Yes      5.00    40

In [25]:
grouped.agg({"tip_pct": ["min", "max", "mean", "std"],
            "size": "sum"})

tip_pct                               size
                  min       max      mean       std  sum
day  smoker                                             
Fri  No      0.120385  0.187735  0.151650  0.028123    9
     Yes     0.103555  0.263480  0.174783  0.051293   31
Sat  No      0.056797  0.291990  0.158048  0.039767  115
     Yes     0.035638  0.325733  0.147906  0.061375  104
Sun  No      0.059447  0.252672  0.160113  0.042347  167
     Yes     0.065660  0.710345  0.187250  0.154134   49
Thur No      0.072961  0.266312  0.160298  0.038774  112
     Yes     0.090014  0.241255  0.163863  0.039389   40

### Returning Aggregated Data Without Row Indexes

In [26]:
tips.groupby(["day", "smoker"], as_index=False).mean()

/var/folders/wh/l_q69dcj75z0c82jy8ry71f40000gn/T/ipykernel_16188/3417973839.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  tips.groupby(["day", "smoker"], as_index=False).mean()


,day,smoker,total_bill,tip,size,tip_pct
0,Fri,No,18.420000,2.812500,2.250000,0.151650
1,Fri,Yes,16.813333,2.714000,2.066667,0.174783
2,Sat,No,19.661778,3.102889,2.555556,0.158048
3,Sat,Yes,21.276667,2.875476,2.476190,0.147906
4,Sun,No,20.506667,3.167895,2.929825,0.160113
5,Sun,Yes,24.120000,3.516842,2.578947,0.187250
6,Thur,No,17.113111,2.673778,2.488889,0.160298
7,Thur,Yes,19.190588,3.030000,2.352941,0.163863


# 10.3 Apply: General split-apply-combine

In [27]:
def top(df, n=5, column="tip_pct"):
    return df.sort_values(column, ascending=False)[:n]

In [28]:
top(tips, n=6)

,total_bill,tip,smoker,day,time,size,tip_pct
172,7.25,5.15,Yes,Sun,Dinner,2,0.710345
178,9.60,4.00,Yes,Sun,Dinner,2,0.416667
67,3.07,1.00,Yes,Sat,Dinner,1,0.325733
232,11.61,3.39,No,Sat,Dinner,2,0.291990
183,23.17,6.50,Yes,Sun,Dinner,4,0.280535
109,14.31,4.00,Yes,Sat,Dinner,2,0.279525


In [29]:
tips.groupby("smoker").apply(top)

total_bill   tip smoker   day    time  size   tip_pct
smoker                                                           
No     232       11.61  3.39     No   Sat  Dinner     2  0.291990
       149        7.51  2.00     No  Thur   Lunch     2  0.266312
       51        10.29  2.60     No   Sun  Dinner     2  0.252672
       185       20.69  5.00     No   Sun  Dinner     5  0.241663
       88        24.71  5.85     No  Thur   Lunch     2  0.236746
Yes    172        7.25  5.15    Yes   Sun  Dinner     2  0.710345
       178        9.60  4.00    Yes   Sun  Dinner     2  0.416667
       67         3.07  1.00    Yes   Sat  Dinner     1  0.325733
       183       23.17  6.50    Yes   Sun  Dinner     4  0.280535
       109       14.31  4.00    Yes   Sat  Dinner     2  0.279525

In [30]:
tips.groupby(["smoker", "day"]).apply(top, n=1, column="total_bill")

total_bill    tip smoker   day    time  size   tip_pct
smoker day                                                             
No     Fri  94        22.75   3.25     No   Fri  Dinner     2  0.142857
       Sat  212       48.33   9.00     No   Sat  Dinner     4  0.186220
       Sun  156       48.17   5.00     No   Sun  Dinner     6  0.103799
       Thur 142       41.19   5.00     No  Thur   Lunch     5  0.121389
Yes    Fri  95        40.17   4.73    Yes   Fri  Dinner     4  0.117750
       Sat  170       50.81  10.00    Yes   Sat  Dinner     3  0.196812
       Sun  182       45.35   3.50    Yes   Sun  Dinner     3  0.077178
       Thur 197       43.11   5.00    Yes  Thur   Lunch     4  0.115982

In [31]:
result = tips.groupby("smoker")["tip_pct"].describe()
result

,count,mean,std,min,25%,50%,75%,max
smoker,,,,,,,,
No,151.0,0.159328,0.039910,0.056797,0.136906,0.155625,0.185014,0.291990
Yes,93.0,0.163196,0.085119,0.035638,0.106771,0.153846,0.195059,0.710345


In [32]:
result.unstack("smoker")

       smoker
count  No        151.000000
       Yes        93.000000
mean   No          0.159328
       Yes         0.163196
std    No          0.039910
       Yes         0.085119
min    No          0.056797
       Yes         0.035638
25%    No          0.136906
       Yes         0.106771
50%    No          0.155625
       Yes         0.153846
75%    No          0.185014
       Yes         0.195059
max    No          0.291990
       Yes         0.710345
dtype: float64

In [77]:
# def f(group):
#     return group.decribe()
    
# grouped.apply(f)

### Suppressing the Group Keys

In [33]:
tips.groupby("smoker", group_keys=False).apply(top)

,total_bill,tip,smoker,day,time,size,tip_pct
232,11.61,3.39,No,Sat,Dinner,2,0.291990
149,7.51,2.00,No,Thur,Lunch,2,0.266312
51,10.29,2.60,No,Sun,Dinner,2,0.252672
185,20.69,5.00,No,Sun,Dinner,5,0.241663
88,24.71,5.85,No,Thur,Lunch,2,0.236746
172,7.25,5.15,Yes,Sun,Dinner,2,0.710345
178,9.60,4.00,Yes,Sun,Dinner,2,0.416667
67,3.07,1.00,Yes,Sat,Dinner,1,0.325733
183,23.17,6.50,Yes,Sun,Dinner,4,0.280535
109,14.31,4.00,Yes,Sat,Dinner,2,0.279525


### Quantile & Bucket Analysis

In [34]:
frame = pd.DataFrame({"data1": np.random.standard_normal(1000),
                     "data2": np.random.standard_normal(1000)})

In [35]:
frame.head()

,data1,data2
0,1.433527,-0.291900
1,-1.380570,1.379606
2,-0.444112,-0.276094
3,1.116347,-0.313755
4,0.201190,0.021770


In [36]:
quartiles = pd.cut(frame["data1"], 4)

In [37]:
quartiles.head()

0     (0.0673, 1.874]
1    (-1.739, 0.0673]
2    (-1.739, 0.0673]
3     (0.0673, 1.874]
4     (0.0673, 1.874]
Name: data1, dtype: category
Categories (4, interval[float64, right]): [(-3.552, -1.739] < (-1.739, 0.0673] < (0.0673, 1.874] < (1.874, 3.68]]

In [38]:
def get_stats(group):
    return pd.DataFrame(
    {"min": group.min(), "max": group.max(),
    "count": group.count(), "mean": group.mean()}
    )

In [39]:
grouped = frame.groupby(quartiles)

In [40]:
grouped.apply(get_stats)

min       max  count      mean
data1                                                      
(-3.552, -1.739] data1 -3.545051 -1.743947     40 -2.076926
                 data2 -1.702657  1.778139     40  0.133591
(-1.739, 0.0673] data1 -1.738246  0.066211    492 -0.640685
                 data2 -4.165595  2.753686    492  0.039384
(0.0673, 1.874]  data1  0.067902  1.840877    434  0.742186
                 data2 -2.457770  3.058692    434 -0.066336
(1.874, 3.68]    data1  1.879943  3.679737     34  2.311780
                 data2 -1.413266  2.322569     34  0.220010

In [41]:
#grouped.agg(["min", "max", "count", "min"])

In [42]:
quartiles_samp = pd.qcut(frame["data1"], 4, labels=False)

In [43]:
quartiles_samp.head()

0    3
1    0
2    1
3    3
4    2
Name: data1, dtype: int64

In [44]:
grouped = frame.groupby(quartiles_samp)

In [45]:
grouped.apply(get_stats)

min       max  count      mean
data1                                           
0     data1 -3.545051 -0.705283    250 -1.249617
      data2 -2.806602  2.182464    250  0.094539
1     data1 -0.705190 -0.017053    250 -0.346250
      data2 -4.165595  2.753686    250 -0.005534
2     data1 -0.016686  0.673782    250  0.329596
      data2 -2.457770  3.058692    250 -0.006374
3     data1  0.674855  3.679737    250  1.275933
      data2 -2.334955  2.346156    250 -0.068986

### Example: Filling Missing Values With Group-Specific Values

In [46]:
s = pd.Series(np.random.standard_normal(6))

In [47]:
s[::2] = np.nan

In [48]:
s

0         NaN
1   -0.010755
2         NaN
3   -1.614588
4         NaN
5    0.568195
dtype: float64

In [49]:
s.fillna(s.mean())

0   -0.352383
1   -0.010755
2   -0.352383
3   -1.614588
4   -0.352383
5    0.568195
dtype: float64

In [50]:
states = ["Ohio", "New York", "Vermont", "Florida", "Oregon", "Nevada", "California", "Idaho"]

In [51]:
group_key = ["East", "East", "East", "East", "West", "West", "West", "West"]

In [52]:
data = pd.Series(np.random.standard_normal(8), index=states)
data

Ohio         -1.019271
New York     -0.370319
Vermont       0.031883
Florida      -3.391864
Oregon       -2.451499
Nevada        0.561290
California   -1.392637
Idaho         1.279580
dtype: float64

In [53]:
data[["Vermont", "Nevada", "Idaho"]] = np.nan
data

Ohio         -1.019271
New York     -0.370319
Vermont            NaN
Florida      -3.391864
Oregon       -2.451499
Nevada             NaN
California   -1.392637
Idaho              NaN
dtype: float64

In [54]:
data.groupby(group_key).size()

East    4
West    4
dtype: int64

In [55]:
data.groupby(group_key).count()

East    3
West    2
dtype: int64

In [56]:
data.groupby(group_key).mean()

East   -1.593818
West   -1.922068
dtype: float64

In [57]:
def fill_mean(group):
    return group.fillna(group.mean())
    

In [58]:
data.groupby(group_key).apply(fill_mean)

/var/folders/wh/l_q69dcj75z0c82jy8ry71f40000gn/T/ipykernel_16188/763740747.py:1: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  data.groupby(group_key).apply(fill_mean)


Ohio         -1.019271
New York     -0.370319
Vermont      -1.593818
Florida      -3.391864
Oregon       -2.451499
Nevada       -1.922068
California   -1.392637
Idaho        -1.922068
dtype: float64

In [59]:
fill_values = {"East": 0.5, "West": -1}

In [60]:
def fill_func(group):
    return group.fillna(fill_values[group.name])

In [61]:
data.groupby(group_key).apply(fill_func)

/var/folders/wh/l_q69dcj75z0c82jy8ry71f40000gn/T/ipykernel_16188/2085919508.py:1: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  data.groupby(group_key).apply(fill_func)


Ohio         -1.019271
New York     -0.370319
Vermont       0.500000
Florida      -3.391864
Oregon       -2.451499
Nevada       -1.000000
California   -1.392637
Idaho        -1.000000
dtype: float64

### Example: Random Sampling & Permutation

In [63]:
suits = ["H", "S", "C", "D"] # Heart, Spades, Clubs, Diamonds
card_val = (list(range(1, 11)) + [10] * 3) * 4
base_names = ["A"] + list(range(2, 11)) + ["J", "K", "Q"]
cards = []
for suit in suits:
    cards.extend(str(sum) + suit for num in base_names)

In [67]:
deck = pd.Series(card_val, index=cards)

In [70]:
deck.head(13)

<built-in function sum>H     1
<built-in function sum>H     2
<built-in function sum>H     3
<built-in function sum>H     4
<built-in function sum>H     5
<built-in function sum>H     6
<built-in function sum>H     7
<built-in function sum>H     8
<built-in function sum>H     9
<built-in function sum>H    10
<built-in function sum>H    10
<built-in function sum>H    10
<built-in function sum>H    10
dtype: int64

In [74]:
def draw(deck, n=5):
    return deck.sample(n)

In [75]:
draw(deck)

<built-in function sum>C     3
<built-in function sum>H    10
<built-in function sum>D     2
<built-in function sum>C     9
<built-in function sum>S    10
dtype: int64

In [76]:
def get_suit(card):
    return card[-1]

In [77]:
deck.groupby(get_suit).apply(draw, n=2)

C  <built-in function sum>C     4
   <built-in function sum>C     3
D  <built-in function sum>D     6
   <built-in function sum>D     2
H  <built-in function sum>H     6
   <built-in function sum>H     3
S  <built-in function sum>S    10
   <built-in function sum>S     3
dtype: int64

In [78]:
deck.groupby(get_suit, group_keys=False).apply(draw, n=2)

<built-in function sum>C     3
<built-in function sum>C    10
<built-in function sum>D     1
<built-in function sum>D     7
<built-in function sum>H     7
<built-in function sum>H     5
<built-in function sum>S     7
<built-in function sum>S     9
dtype: int64

### Example: Group Weighthed Average and Correlation

In [80]:
df = pd.DataFrame({"category": ["a", "a", "a", "a", "b", "b", "b", "b"],
               "data": np.random.standard_normal(8),
               "weights": np.random.uniform(size=8)})

In [81]:
df

,category,data,weights
0,a,0.409412,0.683726
1,a,-1.859871,0.393730
2,a,-0.812381,0.576011
3,a,-0.481443,0.655516
4,b,-1.026106,0.468473
5,b,0.610593,0.555863
6,b,0.041935,0.139163
7,b,0.465506,0.064109


In [82]:
grouped = df.groupby("category")

In [83]:
def get_wavg(group):
    return np.average(group["data"], weights=group["weights"])

In [84]:
grouped.apply(get_wavg)

category
a   -0.535255
b   -0.086036
dtype: float64

In [85]:
close_px = pd.read_csv("examples/stock_px.csv", parse_dates=True, index_col=0)

In [86]:
close_px.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2214 entries, 2003-01-02 to 2011-10-14
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   AAPL    2214 non-null   float64
 1   MSFT    2214 non-null   float64
 2   XOM     2214 non-null   float64
 3   SPX     2214 non-null   float64
dtypes: float64(4)
memory usage: 86.5 KB


In [87]:
close_px.tail(4)

,AAPL,MSFT,XOM,SPX
2011-10-11,400.29,27.00,76.27,1195.54
2011-10-12,402.19,26.96,77.16,1207.25
2011-10-13,408.43,27.18,76.37,1203.66
2011-10-14,422.00,27.27,78.11,1224.58


In [88]:
def spx_corr(group):
    return group.corrwith(group["SPX"])

In [89]:
rets = close_px.pct_change().dropna()

In [90]:
def get_year(x):
    return x.year

In [91]:
by_year = rets.groupby(get_year)

In [92]:
by_year.apply(spx_corr)

,AAPL,MSFT,XOM,SPX
2003,0.541124,0.745174,0.661265,1.0
2004,0.374283,0.588531,0.557742,1.0
2005,0.467540,0.562374,0.631010,1.0
2006,0.428267,0.406126,0.518514,1.0
2007,0.508118,0.658770,0.786264,1.0
2008,0.681434,0.804626,0.828303,1.0
2009,0.707103,0.654902,0.797921,1.0
2010,0.710105,0.730118,0.839057,1.0
2011,0.691931,0.800996,0.859975,1.0


In [93]:
def corr_aapl_msft(group):
    return group["AAPL"].corr(group["MSFT"])

In [94]:
by_year.apply(corr_aapl_msft)

2003    0.480868
2004    0.259024
2005    0.300093
2006    0.161735
2007    0.417738
2008    0.611901
2009    0.432738
2010    0.571946
2011    0.581987
dtype: float64

### Example: Group-Wise Linear Regression

In [95]:
import statsmodels.api as sm

In [98]:
def regress(data, yvar=None, xvars=None):
    Y = data[yvar]
    X = data[xvars]
    X["intercept"] = 1.
    result =sm.OLS(Y, X).fit()
    return result.params

In [99]:
by_year.apply(regress, yvar="AAPL", xvars=["SPX"])

,SPX,intercept
2003,1.195406,0.000710
2004,1.363463,0.004201
2005,1.766415,0.003246
2006,1.645496,0.000080
2007,1.198761,0.003438
2008,0.968016,-0.001110
2009,0.879103,0.002954
2010,1.052608,0.001261
2011,0.806605,0.001514


# 10.4 Group Transform & "Unwrapped" GroupBys

In [101]:
df =pd.DataFrame({"key": ['a', 'b', 'c'] * 4,
                  "value": np.arange(12.)})

In [102]:
df

,key,value
0,a,0.0
1,b,1.0
2,c,2.0
3,a,3.0
4,b,4.0
5,c,5.0
6,a,6.0
7,b,7.0
8,c,8.0
9,a,9.0


In [104]:
g = df.groupby('key')['value']

In [105]:
g.mean()

key
a    4.5
b    5.5
c    6.5
Name: value, dtype: float64

In [106]:
def get_mean(group):
    return group.mean()

In [107]:
g.transform(get_mean)

0     4.5
1     5.5
2     6.5
3     4.5
4     5.5
5     6.5
6     4.5
7     5.5
8     6.5
9     4.5
10    5.5
11    6.5
Name: value, dtype: float64

In [108]:
g.transform('mean')

0     4.5
1     5.5
2     6.5
3     4.5
4     5.5
5     6.5
6     4.5
7     5.5
8     6.5
9     4.5
10    5.5
11    6.5
Name: value, dtype: float64

In [109]:
def times_two(group):
    return group * 2

In [110]:
g.transform(times_two)

0      0.0
1      2.0
2      4.0
3      6.0
4      8.0
5     10.0
6     12.0
7     14.0
8     16.0
9     18.0
10    20.0
11    22.0
Name: value, dtype: float64

In [111]:
def get_ranks(group):
    return group.rank(ascending=False)

In [112]:
g.transform(get_ranks)

0     4.0
1     4.0
2     4.0
3     3.0
4     3.0
5     3.0
6     2.0
7     2.0
8     2.0
9     1.0
10    1.0
11    1.0
Name: value, dtype: float64

In [113]:
def normalize(x):
    return (x - x.mean()) / x.std()

In [114]:
g.transform(normalize)

0    -1.161895
1    -1.161895
2    -1.161895
3    -0.387298
4    -0.387298
5    -0.387298
6     0.387298
7     0.387298
8     0.387298
9     1.161895
10    1.161895
11    1.161895
Name: value, dtype: float64

In [115]:
g.apply(normalize)

/var/folders/wh/l_q69dcj75z0c82jy8ry71f40000gn/T/ipykernel_16188/400169470.py:1: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  g.apply(normalize)


0    -1.161895
1    -1.161895
2    -1.161895
3    -0.387298
4    -0.387298
5    -0.387298
6     0.387298
7     0.387298
8     0.387298
9     1.161895
10    1.161895
11    1.161895
Name: value, dtype: float64

In [116]:
g.transform('mean')

0     4.5
1     5.5
2     6.5
3     4.5
4     5.5
5     6.5
6     4.5
7     5.5
8     6.5
9     4.5
10    5.5
11    6.5
Name: value, dtype: float64

In [117]:
normalize = (df['value'] - g.transform('mean')) / g.transform('std')

In [118]:
normalize

0    -1.161895
1    -1.161895
2    -1.161895
3    -0.387298
4    -0.387298
5    -0.387298
6     0.387298
7     0.387298
8     0.387298
9     1.161895
10    1.161895
11    1.161895
Name: value, dtype: float64

# Pivot Tables & Cross-Tabulation

In [119]:
tips.head()

,total_bill,tip,smoker,day,time,size,tip_pct
0,16.99,1.01,No,Sun,Dinner,2,0.059447
1,10.34,1.66,No,Sun,Dinner,3,0.160542
2,21.01,3.50,No,Sun,Dinner,3,0.166587
3,23.68,3.31,No,Sun,Dinner,2,0.139780
4,24.59,3.61,No,Sun,Dinner,4,0.146808


In [120]:
tips.pivot_table(index=["day", "smoker"])

/var/folders/wh/l_q69dcj75z0c82jy8ry71f40000gn/T/ipykernel_16188/566656882.py:1: FutureWarning: pivot_table dropped a column because it failed to aggregate. This behavior is deprecated and will raise in a future version of pandas. Select only the columns that can be aggregated.
  tips.pivot_table(index=["day", "smoker"])


size       tip   tip_pct  total_bill
day  smoker                                          
Fri  No      2.250000  2.812500  0.151650   18.420000
     Yes     2.066667  2.714000  0.174783   16.813333
Sat  No      2.555556  3.102889  0.158048   19.661778
     Yes     2.476190  2.875476  0.147906   21.276667
Sun  No      2.929825  3.167895  0.160113   20.506667
     Yes     2.578947  3.516842  0.187250   24.120000
Thur No      2.488889  2.673778  0.160298   17.113111
     Yes     2.352941  3.030000  0.163863   19.190588

In [122]:
tips.pivot_table(index=["time", "day"], columns="smoker",
                values=["tip_pct", "size"])

size             tip_pct          
smoker             No       Yes        No       Yes
time   day                                         
Dinner Fri   2.000000  2.222222  0.139622  0.165347
       Sat   2.555556  2.476190  0.158048  0.147906
       Sun   2.929825  2.578947  0.160113  0.187250
       Thur  2.000000       NaN  0.159744       NaN
Lunch  Fri   3.000000  1.833333  0.187735  0.188937
       Thur  2.500000  2.352941  0.160311  0.163863

In [123]:
tips.pivot_table(index=["time", "day"], columns="smoker",
                values=["tip_pct", "size"], margins=True)

size                       tip_pct                    
smoker             No       Yes       All        No       Yes       All
time   day                                                             
Dinner Fri   2.000000  2.222222  2.166667  0.139622  0.165347  0.158916
       Sat   2.555556  2.476190  2.517241  0.158048  0.147906  0.153152
       Sun   2.929825  2.578947  2.842105  0.160113  0.187250  0.166897
       Thur  2.000000       NaN  2.000000  0.159744       NaN  0.159744
Lunch  Fri   3.000000  1.833333  2.000000  0.187735  0.188937  0.188765
       Thur  2.500000  2.352941  2.459016  0.160311  0.163863  0.161301
All          2.668874  2.408602  2.569672  0.159328  0.163196  0.160803

In [124]:
tips.pivot_table(index=["time", "smoker"], columns="day",
                values="tip_pct", aggfunc=len, margins=True)

day             Fri   Sat   Sun  Thur  All
time   smoker                             
Dinner No       3.0  45.0  57.0   1.0  106
       Yes      9.0  42.0  19.0   NaN   70
Lunch  No       1.0   NaN   NaN  44.0   45
       Yes      6.0   NaN   NaN  17.0   23
All            19.0  87.0  76.0  62.0  244

In [126]:
tips.pivot_table(index=["time", "size", "smoker"], columns="day",
                 values="tip_pct", fill_value=0)

day                      Fri       Sat       Sun      Thur
time   size smoker                                        
Dinner 1    No      0.000000  0.137931  0.000000  0.000000
            Yes     0.000000  0.325733  0.000000  0.000000
       2    No      0.139622  0.162705  0.168859  0.159744
            Yes     0.171297  0.148668  0.207893  0.000000
       3    No      0.000000  0.154661  0.152663  0.000000
            Yes     0.000000  0.144995  0.152660  0.000000
       4    No      0.000000  0.150096  0.148143  0.000000
            Yes     0.117750  0.124515  0.193370  0.000000
       5    No      0.000000  0.000000  0.206928  0.000000
            Yes     0.000000  0.106572  0.065660  0.000000
       6    No      0.000000  0.000000  0.103799  0.000000
Lunch  1    No      0.000000  0.000000  0.000000  0.181728
            Yes     0.223776  0.000000  0.000000  0.000000
       2    No      0.000000  0.000000  0.000000  0.166005
            Yes     0.181969  0.000000  0.000000  0.158843
       3    No      0.187735  0.000000  0.000000  0.084246
            Yes     0.000000  0.000000  0.000000  0.204952
       4    No      0.000000  0.000000  0.000000  0.138919
            Yes     0.000000  0.000000  0.000000  0.155410
       5    No      0.000000  0.000000  0.000000  0.121389
       6    No      0.000000  0.000000  0.000000  0.173706

### Cross-Tabulations: Crosstab

In [132]:
from io import StringIO

In [128]:
data = """Sample Nationality Handedness
....: 1 USA Right-Handed
....: 1 Japan Left-Handed
....: 1 USA Right-Handed
....: 1 Japan Right-Handed
....: 1 Japan Left-Handed
....: 1 Japan Right-Handed
....: 1 USA Right-Handed
....: 1 USA Left-Handed
....: 1 Japan Right-Handed
....: 1 USA Right-Handed"""


In [133]:
data = pd.read_table(StringsIO(data), sep="\s+")

NameError: name 'StringsIO' is not defined